### 피드 포워드 신경망

* 다층 퍼셉트론(Multi Layer Perceptron): 간단한 퍼셉트론을 구조적으로 확장한 신경망. 많은 퍼셉트론이 있는 층을 여러 개 쌓아올린 구조.
* CNN: 디지털 신호 처리에 사용하는 윈도우 필터(window filter)에 영향을 받아 만든 신경망. 컴퓨터 비전에 적합하고 단어나 문장 같은 순차 데이터에서 부분 구조를 감지하는 데도 이상적.


# MLP
* 기본적인 신경망 구성 요소
* 입력으로 데이터 벡터를 받고 출력값 하나를 계산

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

seed = 1337
torch.manual_seed(seed) # 파이토치에서 random seed를 고정하기 위한 함수로 manual_seed 제공
torch.cuda.manual_seed_all(seed) # gpu(2개 이상) 사용할 때 random 값 생성 

# print(torch.rand(5))

# torch.manual_seed(seed)
# print(torch.rand(5))

# torch.manual_seed(seed)
# print(torch.rand(5))

In [10]:
# mlp class
class MultiLayerPerceptron(nn.Module) :
    def __init__(self, input_dim:int, hidden_dim:int, output_dim:int) :
        """
        Args:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x_in, apply_softmax=False) :
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))  # 첫 번째 linear 층에 렐루 활성화 함수가 적용됨 
                                               # 한 층의 출력 개수는 다음 층의 입력 개수와 같아야 함
                                               # 두 linear층 사이에 놓인 비선형 활성화 함수는 필수 --> 복잡한 패턴을 모델링할 수 없음
        output = self.fc2(intermediate) 
        
        if apply_softmax:
            output = F.softmax(output, dim=1) 
        return output

In [11]:
# mlp 객체 생성
batch_size = 2
input_dim = 3
hidden_dim = 100
output_dim = 4

mlp = MultiLayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultiLayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


In [20]:
def describe(x) :
    print("---" * 30)
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))
    
# 입력 데이터
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

# 첫 번째 linear 층 출력
fc1 = nn.Linear(input_dim, hidden_dim)
intermediate = fc1(x_input)
describe(intermediate)

# linear output에 활성화함수 적용값 출력
intermediate = F.relu()

# 두 번째 linear층 출력
fc2 = nn.Linear(hidden_dim, output_dim)
output = fc2(intermediate)
describe(output)

------------------------------------------------------------------------------------------
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.9989, 0.4964, 0.4362],
        [0.7803, 0.3707, 0.1563]])
------------------------------------------------------------------------------------------
타입: torch.FloatTensor
크기: torch.Size([2, 100])
값: 
tensor([[ 0.8093, -1.0344,  0.6736,  0.0363,  0.8183,  0.2733,  0.5725,  0.0081,
          0.8957,  1.1066,  0.5748,  0.0803,  0.1496, -0.0837,  0.6955, -0.8565,
          1.3087,  1.0789, -0.2564,  0.1304, -0.2649,  0.3614,  0.1451, -0.2292,
         -0.3187,  0.2484, -0.0904, -0.1061,  0.2381,  0.0687, -0.3042, -0.2283,
         -0.5808,  0.4489, -0.2671, -0.0181,  0.1161, -0.9804,  0.2172,  0.4448,
         -0.6551,  0.3006,  0.3126, -0.4494,  0.2932, -0.7768, -0.6186, -0.5880,
         -0.1581, -0.4465, -0.7421, -0.2253, -0.2683, -0.0608, -0.1055, -0.2919,
          0.3970, -0.6066,  0.4848, -0.8993,  0.8466,  0.7402, -0.4736, -0.7543,
 